In [1]:
N = ord("V") % 5 + 1
N

2

In [1]:
pip install pandas opencv-python pillow

     ---------------------------------------- 38.8/38.8 MB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import cv2
from PIL import Image, ImageDraw

# Завантаження даних із Excel
file_path_excel = "lab6.xlsx"  # Вкажіть правильний шлях до файлу Excel
variant_number = 2  # Ваш номер варіанту

# Завантаження Excel та вибір потрібного рядка
df = pd.read_excel(file_path_excel)
variant_data = df[df['N'] == variant_number].iloc[0]

# Зображення для обробки
image_path = variant_data['file name']  # Назва файлу з таблиці
image_size = tuple(map(int, variant_data['image size'].split('x')))  # Розмір зображення
glasses_color = variant_data['glasses color']  # Колір окулярів
line_width = int(variant_data['line width'])  # Товщина лінії

# Завантаження зображення
image = cv2.imread(image_path)

# Зменшення зображення
image_resized = cv2.resize(image, image_size, interpolation=cv2.INTER_AREA)

# Конвертуємо зображення для роботи з PIL
image_pil = Image.fromarray(cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB))
draw = ImageDraw.Draw(image_pil)

# Завантаження каскаду для обличчя та очей
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Виявлення обличчя
gray_image = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Малювання окулярів на очах
for (x, y, w, h) in faces:
    roi_gray = gray_image[y:y + h, x:x + w]
    roi_color = image_resized[y:y + h, x:x + w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex, ey, ew, eh) in eyes:
        # Координати очей
        center_x, center_y = x + ex + ew // 2, y + ey + eh // 2
        radius = max(ew, eh) // 2
        # Малюємо коло
        draw.ellipse(
            (center_x - radius, center_y - radius, center_x + radius, center_y + radius),
            outline=glasses_color, width=line_width
        )

# Збереження результату
output_path = "output_with_glasses.jpg"
image_pil.save(output_path)
print(f"Результат збережено у файл: {output_path}")

Результат збережено у файл: output_with_glasses.jpg
